   Author: Ankit Kariryaa, University of Bremen

In [ ]:
import rasterio                  # I/O raster data (netcdf, height, geotiff, ...)
import rasterio.warp             # Reproject raster samples
from rasterio import windows
import fiona                     # I/O vector data (shape, geojson, ...)
import pyproj                    # Change coordinate reference system
import geopandas
import shapely
from shapely.geometry import Point
import imgaug as ia
from imgaug import augmenters as iaa

import numpy as np               # numerical array manipulation
import os
from tqdm import tqdm
from PIL import Image
from itertools import product
from tensorflow.keras.models import load_model

from skimage.segmentation import mark_boundaries
from sklearn.model_selection import train_test_split


%matplotlib inline
import matplotlib.pyplot as plt  # plotting tools


import warnings                  # ignore annoying warnings
warnings.filterwarnings("ignore")
import logging
logger = logging.getLogger()
logger.setLevel(logging.CRITICAL)

%reload_ext autoreload
%autoreload 2
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
def get_tiles(ds, width=256, height=256):
    nols, nrows = ds.meta['width'], ds.meta['height']
    offsets = product(range(0, nols, width), range(0, nrows, height))
    big_window = windows.Window(col_off=0, row_off=0, width=nols, height=nrows)
    for col_off, row_off in  offsets:
        window =windows.Window(col_off=col_off, row_off=row_off, width=width, height=height).intersection(big_window)
        transform = windows.transform(window, ds.transform)
        yield window, transform


In [ ]:
def split_image(base_dir, fn, out_path, out_fn, tile_width=5000, tile_height=5000):
    with rasterio.open(os.path.join(base_dir, fn)) as inds:
        meta = inds.meta.copy()
        for window, transform in get_tiles(inds, tile_width, tile_height ):
            print(window)
            meta['transform'] = transform
            meta['width'], meta['height'] = window.width, window.height
            outpath = os.path.join(out_path,out_fn.format(int(window.col_off), int(window.row_off)))
            with rasterio.open(outpath, 'w', **meta) as outds:
                outds.write(inds.read(window=window))

In [ ]:
base_dir = './'
ndvi_fn = ''
pan_fn = ''

out_path = './cutouts'
ndvi_output_fn = 'ndvi_{}-{}.tif'
pan_output_fn = 'pan_{}-{}.tif'
# os.mkdir(out_path)

In [ ]:
split_image(base_dir, ndvi_fn, out_path, ndvi_output_fn)
split_image(base_dir, pan_fn, out_path, pan_output_fn)